In [ ]:
import numpy as np
import pandas as pd
import scipy 
import sklearn
from collections import Counter
from sklearn.metrics import multilabel_confusion_matrix
from scipy import spatial

In [ ]:
data = pd.read_csv(r"/data.csv",header=None)

In [ ]:
labels = pd.read_csv(r"/label.csv",names=['label'],header=None)

In [ ]:
train_data = data.iloc[:8000]
test_data = data.iloc[8000:]
train_labels = labels.iloc[:8000]
test_labels = labels.iloc[8000:]

In [ ]:
class KMeans:
    
    def calculate_SSE(self, centroid_value_dict, centroid_dict,data):
        #centroid_value_dict - dictionary of centroids
        #centroid_dict - dict of centroids keys and data points indexes
        sse_data = 0
        for i in centroid_dict:
            sse_cluster = 0
            # np.sum()
            for j in centroid_dict[i]:
                dp = list(data.iloc[int(j)])
                for a,b in zip(centroid_value_dict[i],dp):
                    sse_cluster += (a-b)**2
            sse_data+=sse_cluster
        return sse_data    
    
    def Initialize_Centroids(self,data,K):
        m = data.shape[0]
        centroid_value_dict={}
        for i in range(K):
            r = np.random.randint(0, m-1)
            centroid_value_dict[i] = data.iloc[r]
        return centroid_value_dict
        #return centroid_list,centroid_dict
    
    def jaccard_similarity(self,centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

    def train_Kmeans(self,data,K,max_iter=20,mode=1,tol=10):
        #Mode = 1 => Euclidean np.linalg.norm(x-list(data.iloc[i,:]))
        #Mode = 2 => Jaccard
        #Mode = 3 => Cosine
        centroid_value_dict = self.Initialize_Centroids(data,K)
        new_centroid_value_dict = {}
        count = 0
        centroid_dict = {}
        convergence = False
        while((count<max_iter) and not convergence):
            
            for i in list(centroid_value_dict.keys()):
                centroid_dict[i]=[]
            for i in range(data.shape[0]):
                x = data.iloc[i]
                if mode==1 :
                    distance_measure = [np.linalg.norm(x-centroid_value_dict[j])  for j in centroid_value_dict]
                    idx = np.argmin(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==2 :
                    distance_measure = [self.jaccard_similarity(list(x),centroid_value_dict[j]) for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==3 :
                    distance_measure = [1-scipy.spatial.distance.cosine(x,list(centroid_value_dict[j]))  for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                
                prev_centroids=dict(centroid_value_dict)
                
            
            for i in centroid_dict:
                if len(centroid_dict[i]):
                    # print(centroid_dict[i])
                    dps_centroid = centroid_dict[i]
                    centroid_value_dict[i] = np.average(data.iloc[dps_centroid],axis=0)
                    #new_centroid = np.zeros(shape = (data.shape[1],))
                    # for j in (temp_dict[i]).astype('int'):
                    #     new_centroid = [new_centroid[i]+list(data.iloc[j,:])[i] for i in range(0,len(list(new_centroid)))]
                    # new_centroid = [int(c/len(temp_dict[i])) for c in new_centroid]
                    
                # print(i)
            
            
            current_tol=-1
            for i in centroid_value_dict:
                prev_centroid_point = prev_centroids[i]
                new_centroid_point = centroid_value_dict[i]
                change = np.sum(np.absolute(new_centroid_point-prev_centroid_point))
                current_tol = max(change, current_tol)
                
            print("Iteration ",count,": ",current_tol)
                
                # lst=[]
                # for j in range(0,len(list(centroid_value_dict[i]))):
                #     if centroid_value_dict[i][j]!=0:
                #         # dummy = (centroid_value_dict[i])
                #         lst.append((int(new_centroid_value_dict[i][j])-centroid_value_dict[i][j])/centroid_value_dict[i][j])*100
                #     else:
                #         lst.append(0)
                # g += np.sum(lst)/len(new_centroid_value_dict[i])
            # change = g/len(new_centroid_value_dict)
            # if change<10:
            #     break
            # centroid_value_dict =  new_centroid_value_dict
            
            count+=1
            if (current_tol<10):
                convergence = True
                break
           # print("KMeans Iteration",count)
        return centroid_value_dict,centroid_dict
    

In [ ]:
def predict_cluster_labels(C, S, labels):
    '''
    Input : C -> Centroids
            S -> Set of Indicies corresponding to Centroid C
            data -> Data used to form clusters
    Output : Returns an array of size K having labels based on majority voting in the cluster
    '''
    cluster_labels = np.zeros(10,dtype=int)
    for c in C:
        labels_of_points = []
        for point in S[c]:
            labels_of_points.extend(labels.iloc[point])
        counter = Counter(labels_of_points)
        try:
            cluster_labels[c] = max(counter, key=counter.get)
        except:
            cluster_labels[c] = np.random.randint(0,9)
    return cluster_labels

In [ ]:
def jaccard_similarity(centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

In [ ]:
def accuracy(centroids, centroid_Labels, test_data, true_labels, mode=1):
    y_true = list(true_labels['label']);
    y_pred = []
    for index in range(test_data.shape[0]):
        featureset = test_data.iloc[index]
        if mode==1:
            distances = [np.linalg.norm(featureset - centroids[centroid]) for centroid in centroids]
            classification = distances.index(min(distances))
            y_pred.append(centroid_Labels[classification])
        elif mode==3:
            similarity = [1 - spatial.distance.cosine(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification])
        elif mode==2:
            similarity = [jaccard_similarity(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification])
    denominator = test_data.shape[0]
    correctly_classified = 0
    for i in range(0,len(y_pred)):
        if y_true[i] == y_pred[i]:
            correctly_classified += 1
    accuracy = correctly_classified/denominator
    return accuracy

In [ ]:
model1 = KMeans()
centroids1,clusters1 = model1.train_Kmeans(data,10, max_iter=100,mode=1)
Euclidean_SSE = model1.calculate_SSE(centroids1,clusters1,data)
print("Euclidean SSE for the dataset is:",Euclidean_SSE)

Iteration  0 :  24471.586572438166
Iteration  1 :  4388.169079515989
Iteration  2 :  4040.388509377869
Iteration  3 :  3533.597619592584
Iteration  4 :  3569.8033340677093
Iteration  5 :  2083.96049958738
Iteration  6 :  954.6023243919072
Iteration  7 :  850.1320945207674
Iteration  8 :  750.9980112591961
Iteration  9 :  542.9392821231057
Iteration  10 :  526.7875635439364
Iteration  11 :  458.3710273805158
Iteration  12 :  502.7054197875092
Iteration  13 :  411.3728179551121
Iteration  14 :  326.2996271010213
Iteration  15 :  266.58329223730743
Iteration  16 :  289.3805508055078
Iteration  17 :  265.14079960001607
Iteration  18 :  211.3367203443604
Iteration  19 :  205.78938081693215
Iteration  20 :  284.563022817668
Iteration  21 :  287.9812506871908
Iteration  22 :  180.35506230529603
Iteration  23 :  134.62253134796242
Iteration  24 :  136.35144184613173
Iteration  25 :  121.90191851657978
Iteration  26 :  105.16645649432544
Iteration  27 :  113.73765727724404
Iteration  28 :  153.

In [ ]:
cluster_labels1 = predict_cluster_labels(centroids1,clusters1,labels)
cluster_labels1

array([7, 6, 8, 4, 3, 9, 0, 5, 1, 2])

In [ ]:
Accuracy_Euclidean = accuracy(centroids1,cluster_labels1,test_data,test_labels)
Accuracy_Euclidean

0.6405

In [ ]:
model2 = KMeans()
centroids2,clusters2 = model2.train_Kmeans(data, 10, max_iter=100, mode=2)
Jaccard_SSE = model2.calculate_SSE(centroids2, clusters2, data)
print("Jacard SSE for the dataset is:",Jaccard_SSE)

Iteration  0 :  35441.393584742094
Iteration  1 :  4881.077687376726
Iteration  2 :  2703.0767786350148
Iteration  3 :  1823.6830998296423
Iteration  4 :  2353.159906227106
Iteration  5 :  1594.9570998914223
Iteration  6 :  1331.0235785123964
Iteration  7 :  943.6506911242603
Iteration  8 :  1295.2330476190477
Iteration  9 :  986.3542436755093
Iteration  10 :  946.7687087566221
Iteration  11 :  727.2016814350128
Iteration  12 :  0.0
Jacard SSE for the dataset is: 34364573698.62264


In [ ]:
cluster_labels2 = predict_cluster_labels(centroids2,clusters2,labels)
cluster_labels2

array([1, 4, 7, 4, 7, 8, 0, 1, 3, 5])

In [ ]:
Accuracy_Jaccard = accuracy(centroids2, cluster_labels2,test_data,test_labels)
Accuracy_Jaccard

0.115

In [ ]:
model3 = KMeans()
centroids3,clusters3 = model3.train_Kmeans(data,10,max_iter = 100,mode=3)
Cosine_SSE = model3.calculate_SSE(centroids3,clusters3,data)
print("Cosine SSE for the dataset is:",Cosine_SSE)

Iteration  0 :  32631.48867411658
Iteration  1 :  6011.097207493814
Iteration  2 :  3619.1145239523953
Iteration  3 :  2463.4904009317615
Iteration  4 :  1404.1697091451065
Iteration  5 :  1013.1574584359608
Iteration  6 :  898.8524897997031
Iteration  7 :  863.3270994806065
Iteration  8 :  855.1223855878907
Iteration  9 :  999.8725445598532
Iteration  10 :  793.0892960453416
Iteration  11 :  898.0705800398152
Iteration  12 :  769.001732441326
Iteration  13 :  567.5753508582255
Iteration  14 :  528.2218391353249
Iteration  15 :  412.4727085502962
Iteration  16 :  307.3772685638519
Iteration  17 :  227.12512360648208
Iteration  18 :  184.68844221105502
Iteration  19 :  168.57688079539952
Iteration  20 :  139.9127302837585
Iteration  21 :  148.39675469936907
Iteration  22 :  113.2345574459374
Iteration  23 :  125.3568665287645
Iteration  24 :  150.1460634382634
Iteration  25 :  185.73256620157895
Iteration  26 :  233.80770913711967
Iteration  27 :  224.03047925982221
Iteration  28 :  194

In [ ]:
cluster_labels3 = predict_cluster_labels(centroids3,clusters3,labels)
cluster_labels3

array([1, 8, 0, 7, 6, 3, 4, 0, 8, 1])

In [ ]:
Accuracy_Cosine = accuracy(centroids3, cluster_labels3,test_data,test_labels)
Accuracy_Cosine

0.5765